In [3]:
%matplotlib qt

In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import math
import matplotlib.pyplot as plt

class InvestmentCalculator:
    """Handles the investment calculation logic and plotting."""

    def __init__(self, update_status_callback):
        # Callback function to update the status message in the main application
        self.update_status = update_status_callback

    def calculate(self, amount, rate, years, interest_type, compounding_freq):
        """Calculates the final investment balance."""

        r = rate / 100
        if interest_type == "simple":
            return round(amount * (1 + r * years), 2)
        else:  # compound
            n = {'annually': 1, 'semi-annually': 2, 'quarterly': 4, 'monthly': 12}[compounding_freq]
            return round(amount * math.pow((1 + r / n), n * years), 2)

    def plot_growth(self, amount, rate, years, interest_type, compounding_freq):
        """Plots the growth of investment over time."""
        x_values = list(range(years + 1))
        y_values = []
        r = rate / 100

        # Calculate the total amount for each year
        for year in x_values:
            if interest_type == "simple":
                total = amount * (1 + r * year)
            else:
                n = {'annually': 1, 'semi-annually': 2, 'quarterly': 4, 'monthly': 12}[compounding_freq]
                total = amount * math.pow((1 + r / n), n * year)
            y_values.append(total)

        # Plotting the graph
        plt.figure("Investment Growth")
        plt.plot(x_values, y_values, marker='o')
        plt.title(f"Investment Growth Over {years} Years")
        plt.xlabel("Years")
        plt.ylabel("Total Amount (£)")
        plt.grid(True)
        plt.show()

    def run(self):
        """Executes the investment calculator dialog."""
        try:
            amount = simpledialog.askfloat("Investment Calculator", "Enter the deposit amount (£):", minvalue=0.01)
            if amount is None: return  # Handle cancellation

            rate = simpledialog.askfloat("Investment Calculator", "Enter the interest rate (as a percentage):", minvalue=0.01, maxvalue=100)
            if rate is None: return  # Handle cancellation
            if rate <= 0 or rate > 100:
                raise ValueError("Interest rate must be between 0 and 100")
            
            years = simpledialog.askinteger("Investment Calculator", "Enter the number of years for investment:", minvalue=1)
            if years is None: return  # Handle cancellation       
            if years <= 0 or years > 100:
                raise ValueError("Years must be a positive number less than 100")

            interest_type = simpledialog.askstring("Investment Calculator", "Choose 'simple' or 'compound' interest:").lower()
            if interest_type not in ["simple", "compound"]:
                raise ValueError("Interest type must be 'simple' or 'compound'")
            
            compounding_freq = simpledialog.askstring("Investment Calculator", "Choose compounding frequency (annually, semi-annually, quarterly, monthly):").lower()
            if compounding_freq not in ["annually", "semi-annually", "quarterly", "monthly"]:
                raise ValueError("Invalid compounding frequency")

            # Confirmation before calculation
            confirmation = messagebox.askyesno("Confirm Input", 
                                            f"Amount: £{amount}\nRate: {rate}%\nYears: {years}\nInterest: {interest_type.capitalize()}\n\nProceed with calculation?")
            if not confirmation:
                return

            total = self.calculate(amount, rate, years, interest_type, compounding_freq)
            messagebox.showinfo("Result", f"Investment Details:\nAmount: £{amount}\nRate: {rate}%\nYears: {years}\nInterest: {interest_type.capitalize()}\nCompounding: {compounding_freq.capitalize()}\n\nTotal balance: £{total}")

            if messagebox.askyesno("Plot", "Would you like to see the investment growth plot?"):
                self.plot_growth(amount, rate, years, interest_type, compounding_freq)

        except ValueError as e:
            messagebox.showerror("Error", str(e))
        except Exception as e:
            messagebox.showerror("Error", f"An unexpected error occurred: {e}")

class BondCalculator:
    """Handles the bond repayment calculation logic."""

    def __init__(self, update_status_callback):
        # Callback function to update the status message in the main application
        self.update_status = update_status_callback

    def calculate(self, value, rate, months):
        """Calculates the monthly bond repayment amount."""
        
        i = rate / 100 / 12
        return round((i * value) / (1 - (1 + i) ** -months), 2)

    def run(self):
        """Executes the bond calculator dialog."""

        try:
            value = simpledialog.askfloat("Bond Calculator", "Enter the value of the house (£):", minvalue=0.01)
            if value is None: return  # Handle cancellation

            rate = simpledialog.askfloat("Bond Calculator", "Enter the interest rate (as a percentage):", minvalue=0.01, maxvalue=100)
            if rate is None: return  # Handle cancellation

            months = simpledialog.askinteger("Bond Calculator", "Enter the number of months to repay the bond:", minvalue=1)
            if months is None: return  # Handle cancellation

            if value <= 0:
                raise ValueError("House value must be positive")
            
            if rate <= 0 or rate > 100:
                raise ValueError("Interest rate must be between 0 and 100")
            
            if months <= 0 or months > 600:  # Assuming 50 years as the maximum repayment period
                raise ValueError("Months must be a positive number and realistically repayable")

            # Confirmation before calculation
            confirmation = messagebox.askyesno("Confirm Input", 
                                            f"House Value: £{value}\nRate: {rate}%\nMonths: {months}\n\nProceed with calculation?")
            if not confirmation:
                return

            repayment = self.calculate(value, rate, months)
            messagebox.showinfo("Result", f"Bond Details:\nHouse Value: £{value}\nRate: {rate}%\nMonths: {months}\n\nMonthly repayment: £{repayment}")

        except ValueError as e:
            messagebox.showerror("Error", str(e))
        except Exception as e:
            messagebox.showerror("Error", f"An unexpected error occurred: {e}")

class FinancialCalculatorsApp:
    """Main application class for the financial calculators."""

    def __init__(self):
        """Initializes the main application window and its widgets."""

        self.root = tk.Tk()
        self.root.title("Financial Calculators")
        self.root.geometry("300x300")

        self.frame = tk.Frame(self.root, padx=10, pady=10)
        self.frame.pack(expand=True, fill=tk.BOTH)

        self.status_label = tk.Label(self.root, text="Welcome!", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X)

        self.create_widgets()

    def update_status(self, message):
        """Updates the status message displayed in the application."""

        self.status_label.config(text=message)

    def create_widgets(self):
        """Creates and lays out the widgets in the application."""

        instruction_label = tk.Label(self.frame, text="Please choose a calculator:", font=("Arial", 12))
        instruction_label.pack(pady=(1, 10))

        inv_button = tk.Button(self.frame, text="Investment Calculator", command=lambda: self.update_status("Investment Calculator selected"))
        inv_button.pack(padx=10, pady=5, fill=tk.X)
        inv_button.bind("<Button-1>", lambda e: InvestmentCalculator(self.update_status).run())

        bond_button = tk.Button(self.frame, text="Bond Calculator", command=lambda: self.update_status("Bond Calculator selected"))
        bond_button.pack(padx=10, pady=5, fill=tk.X)
        bond_button.bind("<Button-1>", lambda e: BondCalculator(self.update_status).run())

    def run(self):
        """Starts the main application loop."""
        self.root.mainloop()

if __name__ == "__main__":
    # Entry point of the application
    app = FinancialCalculatorsApp()
    app.run()